In [24]:
from transformers import AutoModel, AutoTokenizer
import torch
import json
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', 
                                  trust_remote_code = True,
                                 use_flash_attention_2 = True, torch_dtype = torch.bfloat16)

[2023-11-26 04:45:30,326] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [5]:
_ = model.cuda()

In [6]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

In [11]:
input_ids = tokenizer(
    [
        'tak suka ayam',
    ], 
    return_tensors = 'pt',
    padding = True
)
v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
v.shape

(1, 1536)

In [14]:
contexts, questions = [], []

with open('ultrachat-crossref-melayu-malay.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            contexts.append(l[0]['content'].strip())
            questions.append(l[1]['content_ms'].strip())
            
len(contexts), len(questions)

(9959, 9959)

In [17]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    input_ids = tokenizer([contexts[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    contexts_v.append(v)

100%|██████████| 9959/9959 [05:31<00:00, 30.07it/s]


In [18]:
questions_v = []
for i in tqdm(range(len(questions))):
    input_ids = tokenizer([questions[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    questions_v.append(v)

100%|██████████| 9959/9959 [00:50<00:00, 197.38it/s]


In [23]:
contexts_v_np = np.array(contexts_v)[:,0]
questions_v_np = np.array(questions_v)[:,0]

In [25]:
contexts_v_np.shape, questions_v_np.shape

((9959, 1536), (9959, 1536))

In [26]:
contexts_v_np[0]

array([ 0.01206549,  0.04145329, -0.04369389, ..., -0.02253767,
        0.01490884, -0.01677056], dtype=float32)

In [47]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 9959/9959 [04:37<00:00, 35.88it/s]


In [48]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.06777788934631991
3 0.142584596847073
5 0.18817150316296816
10 0.25715433276433375
